<a href="https://colab.research.google.com/github/Meghana-krishna-simma/Business-Decessions/blob/main/Copy_of_Breast_Cancer_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = ':https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F596543%2F1073067%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240320%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240320T190821Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4e13313de7c33a850fea601c7a5f989456e36c27f8f4ef46117d850f36a62df02cde7cfb1aaca7f817b7d1f6527b59be12fba4e188431f83b3f2018593b60f6189994efabc750483b0f5b47c64de48578ce7eeab112105cf68b3fd8ab9ec623412b0cda3efdc7c6a15f9d3d952a0ce4e4bbd955712f6dabe83f1d883e0c0ef07e2700d4e6c184d2a750860f1a00aa01cd2d229534512d838e0517ce0dd26175a83762c2d11ed8b1278e19505bff4d3d498fcbb8cff7d1b538c7df912a11693a1fc8f2d6bf604829856955d3321498a4ef747003fa24631ba99e2715643127c83fb8e46338c344126e92618819532c64e8327c225ca77956996a651752468bf96'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Problem Statement

Breast Cancer is one of the leading cancer developed in many countries including India.With early diagnosis 97% women can survive for more than 5 years. Statistically, the death toll due to this disease has increased drastically in last few decades. If we recognise early, we can take the necesaary action which results in less death toll. Hence,apart from medicinal solutions some Data Science solution needs to be integrated for resolving the death causing issue.

# Objective:
To Build a breast cancer classifier on the dataset that can be accurately classify as benign and malignant.

# Importing the Packages

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Loading the dataset.

In [ ]:
cell_df=pd.read_csv("/kaggle/input/cell_samples.csv")

# About the dataset.

When pathologists examine FNA(fine needle aspirate) tissues samples in breast cancer diagnosis,they consider the nine attributes.each of the attribute is assigned to number from 1-10 by the pathologists.the larger the number the
greater the likelihood of malignancy.no single measurement can be used to determine whether it is benign or malignant.

This is the description of the features of the dataset.

1. Sample code number: id number
2. Clump Thickness: 1 - 10
3. Uniformity of Cell Size: 1 - 10
4. Uniformity of Cell Shape: 1 - 10
5. Marginal Adhesion: 1 - 10
6. Single Epithelial Cell Size: 1 - 10
7. Bare Nuclei: 1 - 10
8. Bland Chromatin: 1 - 10
9. Normal Nucleoli: 1 - 10
10. Mitoses: 1 - 10
11. Class: (2 for benign, 4 for malignant)

Clump thickness indicates that radius was computed by averaging the length of radial line segments from the center of
the nuclear mass to each of the points of the nuclear border. For cell size, perimeter was measured as the distance
around the nuclear border which is considered to be uniform. For measuring the cell shape, area is measured by
counting the number of pixels in the interior of the nuclear border and adding one-half of the pixels on the perimeter.
Marginal adhesion is measured bycombining the perimeter and area to give a measure of the compactness of the cell
nuclei

#classify cells to whether the samples are benign(mild state)=2 or malignant(evil state)=4

In [ ]:
# There are 699 records and for each record we have multiple parameters that is measured.
# Public Source - https://s3-api.us-geo.objectstorage.s... "
# we are using a dataset that has a 9 predictors in each record, 699 records

In [ ]:
cell_df.head()

In [ ]:
cell_df.tail()

* ‘id’, ‘clump thickness’, ‘uniformity of cell size’, ‘uniformity of cell shape’, ‘marginal adhesion’, ‘single epithelial cell size’, ‘bare nuclei’, ‘bland chromatin’, ‘normal nucleoli’, ‘mitosis’ are the variables used to predict the output ‘class’.

# Dropping the unwanted columns.

In [ ]:
# Let us drop the ID column as it doesnot influence the output "class".
cell_df.drop('ID',axis=1,inplace=True)

In [ ]:
cell_df.shape

The data frame is of shape (699,10) suggesting there are 699 training cases.

# Checking for missing values

In [ ]:
#Missing Or Null data points

cell_df.isnull().sum()
cell_df.isna().sum()

No Missing values found in the dataset.

In [ ]:
cell_df.count()

The count of each column is 699 which suggests there are no missing values.

# Checking for Categorical variables

In [ ]:
cell_df.columns

In [ ]:
col_names = ['Clump', 'UnifSize', 'UnifShape', 'MargAdh', 'SingEpiSize', 'BareNuc', 'BlandChrom', 'NormNucl', 'Mit', 'Class']
for x in col_names:
    print(cell_df[x].nunique())

All the variables are Categorical variables.

# Checking whether the dataset is a balanced or not.

In [ ]:
# Let us check whether the dataset is a balanced or imbalanced one.
target_count = cell_df.Class.value_counts()
print('Benign:', target_count[2])
print('Malignant:', target_count[4])

In [ ]:
458/241

It is a balanced dataset. the proportion of benign and malignant is almost 2:1.

In [ ]:
cell_df.dtypes

BareNuc variable is an object data type, so need to convert it into integer data type.

Bare Buc is an object type which is not a numeric value. So, we cannot apply mathematical operations on this column.
So, Here lets simply removethe non numeric data.
For a particular row, and in that row a particular column( BareNuc) if values are non numeric. We could simply remove it, as synthetic data would not contribute to right decisions.

In [ ]:
# Identify the unwanted rows

cell_df = cell_df[pd.to_numeric(cell_df['BareNuc'],errors='coerce').notnull()]
cell_df['BareNuc']=cell_df['BareNuc'].astype('int')

In [ ]:
cell_df.dtypes

So, BareNuc  is converted to integer data type.

In [ ]:
cell_df.shape

# Summary

In [ ]:
cell_df.describe().transpose()

The output variable ‘class’ is discrete and takes two values :- 2 (Benign) and 4 (Malignant).
The mean of ‘class’ is closer to 2 indicating there are more benign cases.
The minimum and maximum value of all input variables are 1 and 10 respectively.

# Data Visualisation

In [ ]:
cell_df.hist(figsize=(20,12))
plt.show()

‘clump thickness’ is evenly distributed to some extent. All other variables are skewed to the right.

In [ ]:
cell_df.columns

# Taking out the predictors and predicted variables seperately

In [ ]:
# Taking out the predictors and predicted variables seperately for the further.

feature_df=cell_df[['Clump', 'UnifSize', 'UnifShape', 'MargAdh', 'SingEpiSize',
       'BareNuc', 'BlandChrom', 'NormNucl', 'Mit']]
x=np.asarray(feature_df)
y=np.asarray(cell_df['Class'])


In [ ]:
x[0:5]

# Splitting the dataset.

In [ ]:
# Divide the data as train and test dataset

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=4)
x_train.shape

# Training My Models

# Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(C=100,random_state=0)
lr.fit(x_train,y_train)
y_pred=lr.predict(x_test)
correct = (y_test == y_pred).sum()
incorrect = (y_test != y_pred).sum()
accuracy = correct / (correct + incorrect) * 100

print('\nPercent Accuracy: %0.1f' %accuracy)

In [ ]:
prediction = pd.DataFrame()
prediction['actual'] = y_test
prediction['predicted'] = y_pred
prediction['correct'] = prediction['actual'] == prediction['predicted']

print ('\nDetailed results for first 20 tests:')
print (prediction.head(20))


In [ ]:
#Accuracy of our model.
from sklearn.metrics import confusion_matrix
c_logistic=confusion_matrix(y_test,y_pred)
print(c_logistic)
Accuracy_logistic=sum(np.diag(c_logistic))/(np.sum(c_logistic))
Accuracy_logistic

In [ ]:
#Evaluation
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

The metrics for this model are as follows
Accuracy : 97.08 %,
Precision : 96%,
Recall: 98%,
F1 Score: 97%.

# Naive Bayes Model

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier_naive=GaussianNB()
classifier_naive.fit(x_train, y_train)
y_predict=classifier_naive.predict(x_test)

In [ ]:
#Accuracy of our model.
from sklearn.metrics import confusion_matrix
c_naive=confusion_matrix(y_test,y_predict)
print(c_naive)
Accuracy_naive=sum(np.diag(c_naive))/(np.sum(c_naive))
Accuracy_naive

In [ ]:
#Evaluation
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predict))

The metrics for this model are as follows
Accuracy : 94.89 %,
Precision : 94%,
Recall: 96%,
F1 Score: 95%.

# SVM Model

In [ ]:
#modelling SVM

from sklearn import svm
classifier_svm=svm.SVC(kernel='linear',gamma='auto',C=1)
classifier_svm.fit(x_train,y_train)
y_predict=classifier_svm.predict(x_test)

In [ ]:
# Confusion matrix and Accuracy of our model.

from sklearn.metrics import confusion_matrix
c_svm=confusion_matrix(y_test,y_predict)
print(c_svm)
Accuracy_svm=sum(np.diag(c_svm))/(np.sum(c_svm))
Accuracy_svm

In [ ]:
#Evaluation
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predict))

The metrics for this model are as follows
Accuracy : 96.35 %,
Precision : 95%,
Recall: 97%,
F1 Score: 96%.


# Kernal SVM. Kernal taken is polynomial kernal.

In [ ]:
from sklearn import svm
classifier_svmk=svm.SVC(kernel='poly',gamma='auto',C=1)
classifier_svmk.fit(x_train,y_train)
y_predict=classifier_svmk.predict(x_test)

In [ ]:
#Accuracy of our model.
from sklearn.metrics import confusion_matrix
c_svmk=confusion_matrix(y_test,y_predict)
print(c_svmk)
Accuracy_svmk=sum(np.diag(c_svmk))/(np.sum(c_svmk))
Accuracy_svmk

In [ ]:
#Evaluation
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predict))

The metrics for this model are as follows
Accuracy : 97.08 %,
Precision : 96%,
Recall: 98%,
F1 Score: 97%.

# KNN Model

In [ ]:
# modelling Knn Classifier

from sklearn.neighbors import KNeighborsClassifier

# nothing but we are using euclidean distance
classifier_knn=KNeighborsClassifier(n_neighbors=6,metric="minkowski",p=2)
classifier_knn.fit(x_train,y_train)
y_predict=classifier_knn.predict(x_test)

Let us find the optimum value for K. Here we have taken the loop iteration till 27 as per thumb rule optimum k value is always Square root of number of records. so square root of 699 rounds of to 26. So that is why it is taken.

In [ ]:
# lets see the best value of k for which the model is predicting with high accuracy.

n=[]
acc=[]

for i in range(1,27):
    classifier_knn_trail=KNeighborsClassifier(n_neighbors=i,metric="minkowski",p=2)
    classifier_knn_trail.fit(x_train,y_train)
    c_knn_trail=confusion_matrix(y_test,classifier_knn_trail.predict(x_test))
    acc.append(sum(np.diag(c_knn_trail))/(np.sum(c_knn_trail)))
    n.append(i)
n=np.array(n)
acc=np.array(acc)
plt.plot(n,acc)
plt.show()

In [ ]:
#Accuracy of our model.
from sklearn.metrics import confusion_matrix
c_knn=confusion_matrix(y_test,y_predict)
print(c_knn)
Accuracy_knn=sum(np.diag(c_knn))/(np.sum(c_knn))
Accuracy_knn

In [ ]:
#Evaluation
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predict))

The metrics for this model are as follows
Accuracy : 98.54 %,
Precision : 98%,
Recall: 99%,
F1 Score: 98%.

# Decision tree

In [ ]:
# Fitting Decision Tree Classification to the Training set
from sklearn.tree import DecisionTreeClassifier
classifier_tree = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)# for gini 0.948905109489051
classifier_tree.fit(x_train, y_train)

# Predicting the Test set results
y_predict = classifier_tree.predict(x_test)

In [ ]:
#Accuracy of our model.
from sklearn.metrics import confusion_matrix
c_tree=confusion_matrix(y_test,y_predict)
print(c_tree)
Accuracy_tree=sum(np.diag(c_tree))/(np.sum(c_tree))
Accuracy_tree

In [ ]:
#Evaluation
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predict))

The metrics for this model are as follows
Accuracy : 96.35 %,
Precision : 96%,
Recall: 96%,
F1 Score: 96%.

# Random forest

In [ ]:
# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier_ensemble = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier_ensemble.fit(x_train, y_train)

# Predicting the Test set results
y_predict = classifier_ensemble.predict(x_test)

In [ ]:
#Accuracy of our model.
from sklearn.metrics import confusion_matrix
c_ensemble=confusion_matrix(y_test,y_predict)
print(c_ensemble)
Accuracy_ensemble=sum(np.diag(c_ensemble))/(np.sum(c_ensemble))
Accuracy_ensemble

In [ ]:
#Evaluation
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predict))

The metrics for this model are as follows
Accuracy : 97.81 %,
Precision : 97%,
Recall: 98%,
F1 Score: 98%.

# XGBoost

In [ ]:
!pip install xgboost

In [ ]:
# Fitting the XGBoost to the training set
from xgboost import XGBClassifier
classifier_xg=XGBClassifier()
classifier_xg.fit(x_train,y_train)

# Predicting the test results
y_predictor= classifier_xg.predict(x_test)

In [ ]:
# Confusion Matrix
#Accuracy of our model.
from sklearn.metrics import confusion_matrix
c_xg=confusion_matrix(y_test,y_predictor)
print(c_xg)
Accuracy_xg=sum(np.diag(c_xg))/(np.sum(c_xg))
Accuracy_xg

In [ ]:
#Evaluation
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predictor))

The metrics for this model are as follows
Accuracy : 97.81 %,
Precision : 97%,
Recall: 98%,
F1 Score: 98%.

# Model Evaluation Metrics

In [ ]:
d={'Accuracy(%)' : [97.08,94.89,96.35,97.08,98.54,96.35,97.81,97.91],'Precision' : [0.96,0.94,0.95,0.96,0.98,0.96,0.97,0.97],'recall' : [0.98,0.96,0.97,0.98,0.99,0.96,0.98,0.98],'F1 Score' : [0.97,0.95,0.96,0.97,0.98,0.96,0.98,0.98]}
Model_metrics = pd.DataFrame(d,index=['Logistic Regression','Naive Bayes','Svm-Linear','Svm-Polynomial','KNN','Decison Tree','Random Forest','XGBoost'])

In [ ]:
Model_metrics

We can see KNN performing well and then comes random forest and XGBoost which have more accuracy and F1 Score as well.

# Prediction using trained model
The trained models is used to predict a particular case :- ‘clump thickness’ = 1, ‘uniformity of cell size’ = 2, ‘uniformity of cell shape’ = 2, ‘marginal adhesion’ = 5 , ‘single epithelial cell size’ = 3 , 'bare nuclei' = 4 , ‘bland chromatin’ = 6, ‘normal nucleoli’ = 4, ‘mitosis’ = 8.

In [ ]:
y_predict=classifier_knn.predict(np.array([[1,2,2,5,3,4,6,4,8]]))
print(y_predict)

The predicted value of ‘class’ is 2 which suggests it is a benign tumor.

In [ ]:
y_predict=classifier_ensemble.predict(np.array([[1,2,2,5,3,4,6,4,8]]))
print(y_predict)

My random forest model is also predicting the same as benign.

In [ ]:
y_predict=classifier_xg.predict(np.array([[1,2,2,5,3,4,6,4,8]]))
print(y_predict)

My xg boost model is also predicting the same as benign.

# Conclusion
Breast Cancer has become the foremost cause of death worldwide for womens. The most successful way to reduce
cancer deaths is to detect it earlier. Many people avoid cancer screening due to the cost involved in taking numerous
tests for diagnosis.This prediction system may provide easy and a cost effective way for screening cancer and may play
a significant role in earlier diagnosis process for different types of cancer and provide effective preventive
approach.